In [4]:
import pandas as pd
from scipy import stats as st

sales = pd.read_csv('sales data-set.csv')
facts = pd.read_csv('Features data set.csv')

# This just sets us up to start

In [5]:
sales.info()
facts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4  

In [10]:
# Convert the date columns to the correct format in each to ensure later analysis works.

sales['Date'] = pd.to_datetime(sales['Date'], format='%d/%m/%Y')
facts['Date'] = pd.to_datetime(facts['Date'], format='%d/%m/%Y')



In [12]:
# Merge the two dataframes together

merged = pd.merge(sales, facts, on = 'Date')

# Drop any null values to clean things up for later analysis

merged = merged.dropna(subset = ["Temperature", "Weekly_Sales"])

# Then we group by the date and calc the mean of Weekly Sales

sales_calc = merged.groupby('Date')['Weekly_Sales'].mean()

# Create a new dataframe with just the dates, sales and temps.
analysis_df = pd.DataFrame({'Date': sales_calc.index, 'Weekly_Sales': sales_calc.values})
analysis_df = pd.merge(analysis_df, facts[['Date', 'Temperature']], on='Date')

In [13]:
# We can take this new df and calc to see if there's a correlation between temp and sales

correlation = analysis_df['Weekly_Sales'].corr(analysis_df['Temperature'])
print(f"Correlation between sales and temperature: {correlation}")

Correlation between sales and temperature: -0.10730756282193221


In [14]:
# Trying to do a t-test
# First, create high and low temp categories.

median_temp = analysis_df['Temperature'].median()
analysis_df['Temp_Category'] = ['High' if temp > median_temp else 'Low' for temp in analysis_df['Temperature']]

# Perform t-test
high_temp_sales = analysis_df[analysis_df['Temp_Category'] == 'High']['Weekly_Sales']
low_temp_sales = analysis_df[analysis_df['Temp_Category'] == 'Low']['Weekly_Sales']

t_stat, p_value = st.ttest_ind(high_temp_sales, low_temp_sales)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: -8.214634611638882, P-value: 2.550149915407261e-16


So, what we can draw from this is a few things.
There's a slight negative correlation between temp and sales.  The -0.107... tells us that when temp goes up, sales go down, but the correlation being close to zero tells us that this relationship is weak at best.  Just as we found from the general analysis with this data graphing it in matplotlib, temp doesn't really tell us much about sales.

Similarly, as we define a high group and a low group of temp, and do a ttest of these two groups related to sales, we get a high p value, that overall tells us that we ought not reject H0, which in this case would be that temperature change doesn't affect sales.

In [20]:
analysis_df.to_csv('analysis_temp_sales.csv', index = False)